In [14]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf


print(tf.__version__)

2.1.0


In [16]:
xy = np.loadtxt('data/data-03-diabetes.csv', delimiter=',', dtype=np.float32)
x_train = xy[:, 0:-1]
y_train = xy[:, [-1]]

print(x_train.shape, y_train.shape)
print(xy)

# tf data API를 통해 데이터셋을 세팅한다.
# batch size == 한 번에 학습시킬 크기를 의미한다.
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(len(x_train))

W = tf.Variable(tf.random.normal([8, 1]), name='weight')
b = tf.Variable(tf.random.normal([1]), name='bias')

print(W, b)

# optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01) => tf 2.x에서 사용 안 함
optimizer = tf.optimizers.SGD(learning_rate=0.01) # tf 2.x에서 사용하는 Gradient Descent 알고리즘 함수
epochs = 1001

def logistic_regression(features):
    hypothesis = tf.divide(1., 1. + tf.exp(tf.matmul(features, W) + b))
    return hypothesis
    
def cost_fucntion(hypothesis, features, labels):
    cost = tf.reduce_mean(-labels * tf.math.log(logistic_regression(features)) - (1- labels) * tf.math.log(1 - logistic_regression(features)))
    return cost

def accuracy_function(hypothesis, labels):
    predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, labels), dtype=tf.int32))
    return accuracy

def grad(hypothesis, features, labels):
    with tf.GradientTape() as tape:
        loss_value = cost_fucntion(logistic_regression(features), features, labels)
    return tape.gradient(loss_value, [W,b])

for step in range(epochs):
    for features, labels in iter(dataset):
        grads = grad(logistic_regression(features), features, labels)
        optimizer.apply_gradients(grads_and_vars=zip(grads, [W, b]))
        
        if step % 100 == 0:
            print("Iter: {}, Loss: {:.4f}".format(step, cost_fucntion(logistic_regression(features), features, labels)))

(759, 8) (759, 1)
[[-0.294118   0.487437   0.180328  ... -0.53117   -0.0333333  0.       ]
 [-0.882353  -0.145729   0.0819672 ... -0.766866  -0.666667   1.       ]
 [-0.0588235  0.839196   0.0491803 ... -0.492741  -0.633333   0.       ]
 ...
 [-0.411765   0.21608    0.180328  ... -0.857387  -0.7        1.       ]
 [-0.882353   0.266332  -0.0163934 ... -0.768574  -0.133333   0.       ]
 [-0.882353  -0.0653266  0.147541  ... -0.797609  -0.933333   1.       ]]
<tf.Variable 'weight:0' shape=(8, 1) dtype=float32, numpy=
array([[-1.0432005 ],
       [-0.2659193 ],
       [ 1.5240275 ],
       [ 0.39498618],
       [ 0.9954591 ],
       [ 0.7634289 ],
       [-0.35749814],
       [-0.71573275]], dtype=float32)> <tf.Variable 'bias:0' shape=(1,) dtype=float32, numpy=array([1.0567429], dtype=float32)>
Iter: 0, Loss: 1.3726
Iter: 100, Loss: 0.9365
Iter: 200, Loss: 0.7448
Iter: 300, Loss: 0.6695
Iter: 400, Loss: 0.6380
Iter: 500, Loss: 0.6225
Iter: 600, Loss: 0.6132
Iter: 700, Loss: 0.6065
Iter: 8